# Pyomo Summer Workshop 2018 Exercise Problem - Pyomo Fundamentals

This notebook contains material from the Pyomo Summer Workshop 2018 by David Bernal (dbernaln at purdue.edu), Zedong Peng (peng_zedong at 123.com), and Albert Lee (lee4382 at purdue.edu); the content is available on **[Github](https://github.com/AlbertLee125/pyomo-summer-ws)**.
The original sources of this material are available on **[Pyomo](http://www.pyomo.org/workshop-examples)**.
The text is released under the **[CC-BY-NC-ND-4.0](https://creativecommons.org/licenses/by-nc-nd/4.0/legalcode) license, and code is released under the **[MIT license](https://opensource.org/licenses/MIT).*

## Exercise 1: Pyomo Fundamentals
### 1.1. Knapsack example

Solve the knapsack problem shown in tutorial using your IDE (e.g., Spyder) or the command lined: `> python knapsack.py`.
Which items are acquired in the optimal solution?
What is the value of the selected items?

### 1.2. Knapsack with improved printing
The `knapsack.py` in the tutorial uses `model.pprint()` to see the value of the solution variables.
Starting with the code in `knapsack_print_incomplete.py`, complete the missing lines to produce formatted output.
Note that the Pyomo value function should be used to get the floating point value of Pyomo modeling components (e.g., `print(value(model.x[i]))`). 
Also print the value of the items selected (the objective), and the total weight. 
(A solution can be found in `knapsack_print_soln.py`).

### 1.3. Changing data
If we were to increase the value of the wrench, at what point would it become selected as part of the optimal solution?
(A solution can be found in `knapsack_wrench_soln.py`.)

### 1.4. Loading data from Excel 
In the knapsack example shown in the tutorial slides, the data is hardcoded at the top of the file. 
Instead of hardcoding the data, use Python to load the data from a different source.
You can start from the file `knapsack_pandas_excel_incomplete.py`.
(A solution that uses pandas to load the data from Excel is shown in
`knapsack_pandas_excel_soln.py`.)

### 1.5. NLP vs MIP
Solve the knapsack problem with `IPOPT` instead of `glpk`.
(Hint: switch `glpk` to `ipopt` in the call `SolverFactory`.) 
Print the solution values for `model.x`. 
What happened? 
Why?

## Exercise 2: Pyomo Fundamentals
### 2.1. Knapsack problem with rules
Rules are important for defining indexed constraints, however, they can also be used for single (i.e. scalar) constraints. 
Starting with `knapsack.py`, reimplement the model using rules for the objective and the constraints. 
(A solution can be found in `knapsack_rules_soln.py`.)

### 2.2. Integer formulation of the knapsack problem
Consider again, the knapsack problem. 
Assume now that we can acquire multiple items of the same type. 
In this new formulation, $x_i$ is now an integer variable instead of a binary variable. 
One way to formulate this problem is as follows:

$$
\begin{aligned}
\max_{q,x} &~ \sum_{i \in A} v_i x_i \\
\text{s.t.} &~ \sum_{i \in A} w_i x_i \leq W_{\max} \\
&~ x_i - \sum_{j=0}^N j q_{i,j} \quad \forall i \in A \\
&~ 0 \leq x \leq N \\
&~ q_{i,j} \in \{0,1\} \quad~~ \forall i \in A, j \in \{0,1,\dots,N\}
\end{aligned}
$$

Starting with `knapsack_rules.py`, implement this new formulation
and solve. 
Is the solution surprising? 
(A solution can be found in `knapsack_integer_soln.py`.)

## Exercies 3: Pyomo Fundamentals
### 3.1. Using the decorator notation for rules
In the slides, we saw an alternative notation for declaring and defining Pyomo components using decorators. 
Starting with the warehouse location problem in `warehouse_location_decorator_incomplete.py` change the model to use the decorator notation. 
(A solution for this problem can be found in `warehouse_location_decorator_soln.py`.)

### 3.2. Changing Parameter values
In the tutorial slides, we saw that a parameter could be specified to be mutable. 
This tells Pyomo that the value of the parameter may change in the future, and allows the user to
change the parameter value and resolve the problem without the need to rebuild the entire model each time. 
We will use this functionality to find a better solution to an earlier exercise. Considering again the knapsack problem, we would like to find when the wrench becomes valuable enough to be a part of the optimal solution. 
Create a Pyomo Parameter for the value of the items, make it mutable, and then write a loop that prints the solution for different wrench values. 
Start with the file `knapsack_mutable_parameter_incomplete.py`. 
(A solution for this problem can be found in `knapsack_mutable_parameter_soln.py`.)

### 3.3. Integer cuts
Often, it can be important to find not only the "best" solution, but a number of solutions that are equally optimal, or close to optimal. 
For discrete optimization problems, this can be done using something known as an integer cut. 
Consider again the knapsack problem where the choice of which items to select is a discrete variable
$x_i \forall i \in A$. 
Let $x_i^k$ be a particular set of $x$ values we want to remove from the feasible solution space. 
We define an integer cut using two sets. 
The first set $S_0$ contains the indices for those variables whose current solution is $0$, and the second set $S_1$ consists of indices for those variables whose current solution is $1$. 
Given these two sets, an integer cut constraint that would prevent such a solution from appearing again is defined by,

$$
\sum_{i \in S_0} x[i] + \sum_{i \in S_1} (1 - x[i]) \geq 1.
$$

Starting with `knapsack_rules.py`, write a loop that solves the problem $5$ times, adding an integer cut to remove the previous solution, and printing the value of the objective function and the solution at each
iteration of the loop. 
(A solution for this problem can be found in `knapsack_integer_cut_soln.py`.)

### 3.4. Putting it all together with the lot sizing example: (Hart et al., 2017)
We will now write a complete model from scratch using a well-known multi-period optimization problem for optimal lot-sizing adapted from Hagen et al. (2001) shown below.

$$
\begin{align}
\min &~ \sum_{t \in T} c_t y_t + h_t^+ I_t^+ + h_t^- I_t^- \\
\text{s.t.} &~ I_t = I_{t-1} + X_t -d_t \quad \forall t \in T \\
&~ I_t = I_t^+ - I_t^- \quad \quad \quad ~~ \forall t \in T \\
&~ X_t \leq P y_t \quad \quad \quad \quad \quad \forall t \in T \\
&~ X_t, I_t^+, I_t^- \geq 0 \quad \quad \quad  \forall t \in T \\
&~ y_t \in \{0, 1\} \quad \quad \quad \quad ~~ \forall t \in T \\
\end{align}
$$

Our goal is to find the optimal production $X_t$ given known demands $d_t$, fixed cost $c_t$ associated with active production in a particular time period, an inventory holding cost $h^+_t$ and a shortage cost $h_t^-$ (cost of keeping a backlog) of orders. 
The variable $y_t$ (binary) determines if we produce in time $t$ or not, and $I^+_t$ represents inventory that we are storing across time period $t$, while $I^-_t$ represents the magnitude of the backlog.
Note that equation $(4)$ is a constraint that only allows production in time period $t$ if the indicator variable $y_t=1$.

Write a Pyomo model for this problem and solve it using glpk using the data provided below. 
You can start with the file `lot_sizing_incomplete.py`.
(A solution is provided in `lot_sizing_soln.py`.)

<center>

| Parameter | Description                       | Value                 |
| --------- | --------------------------------- | --------------------- |
| $c$         | fixed cost of production          | $4.6$                   |
| $I_0^+$       | initial value of positive inventory | $5.0$                 |
| $I_0^-$       | initial value of backlogged orders | $0.0$                  |
| $h^+$        | cost (per unit) of holding inventory | $0.7$                 |
| $h^-$        | shortage cost (per unit)         | $1.2$                   |
| $P$         | maximum production amount (big-M value) | $5$                 |
| $d$         | demand                            | $[5, 7, 6.2, 3.1, 1.7]$ |

</center>